# Demos: Lecture 6

In [3]:
import pennylane as qml
from pennylane import numpy as np

## Demo 1: teleportation 

 <img src="fig/teleportation.png" width="900px">

In [9]:
def create_entangled_state(wires):
    qml.Hadamard(wires=wires[0])
    qml.CNOT(wires=[wires[0], wires[1]])
    
def prepare_alice_state():
    qml.RX(0.3, wires=0)
    qml.RY(0.5, wires=0)
    qml.Hadamard(wires=0)
    qml.PauliY(wires=0)
    qml.RX(0.1, wires=0)
    

dev = qml.device("default.qubit", wires=3)

@qml.qnode(dev)
def teleport():
    prepare_alice_state()
    
    create_entangled_state([1, 2])
    
    qml.adjoint(create_entangled_state)([0, 1])
    
    m1 = qml.measure(wires=0)
    m2 = qml.measure(wires=1)
    
    qml.cond(m2, qml.PauliX)(wires=2)
    qml.cond(m1, qml.PauliZ)(wires=2)
    
    return qml.probs(wires=2)


@qml.qnode(dev)
def prepare_state_only():
    prepare_alice_state()
    return qml.probs(wires=0)

In [10]:
prepare_state_only()

tensor([0.28688912, 0.71311088], requires_grad=True)

In [11]:
teleport()

tensor([0.28688912, 0.71311088], requires_grad=True)

## Demo 2: expectation values